In [1]:
#Connect to wandb
#TO DO : how to save models on the weight and bias platform
import wandb
wandb.login()
wandb.init(project="svm", 
           entity="benchmark-nlp",
           name='emotion datasets svc') #CHANGE

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jtonglet (benchmark-nlp). Use `wandb login --relogin` to force relogin


In [2]:
import os
os.chdir('../../..')

In [3]:
#Load packages
import warnings
import io
import numpy as np
import pandas as pd
from codecarbon import EmissionsTracker
import yaml
from util.dataloader import DataLoader
from preprocessing.preprocessor import Preprocessor
from util.datasplitter import data_splitter
from preprocessing.fasttext_embeddings import FastTextEmbeddings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC, LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score
warnings.filterwarnings("ignore")

In [4]:
#Set constant values
SEED=42 
OPT_ITER=10  #Change??

## Load data

In [5]:
dl = DataLoader(['emotion'])
data = dl.load()


tweet_preprocessor = Preprocessor(is_tweet=True)
preprocessor = Preprocessor()

#We are not interested in the test sets for hyperparameter optimization
train_eval_emotion, val_eval_emotion, _ = data_splitter(data['tweetEval'],
                                 tweet_preprocessor,  #Need to rerun this one
                                 create_val_set=True,   #No validation set is provided
                                 seed=SEED)
train_carer, val_carer, _ = data_splitter(data['CARER'],
                                 preprocessor, 
                                 create_val_set=True,   #No validation set is provided
                                 seed=SEED)
train_silicone, val_silicone, _ = data_splitter(data['silicone'],
                                 preprocessor, 
                                 create_val_set=True,   #No validation set is provided
                                 seed=SEED)

87170 rows preprocessed in 39.37953495979309 seconds
7740 rows preprocessed in 2.6140074729919434 seconds
8069 rows preprocessed in 2.6870648860931396 seconds


In [9]:
%%time
#fasttext 
fasttext = FastTextEmbeddings()
fasttext.load_model('fasttext/cc.en.300.bin')

CPU times: total: 18 s
Wall time: 34.2 s


In [10]:
embedded_train_eval_emotion = fasttext.generate_sentence_embeddings(train_eval_emotion['text'])
embedded_val_eval_emotion = fasttext.generate_sentence_embeddings(val_eval_emotion['text'])
embedded_train_eval_emotion['label'] = train_eval_emotion['label'].to_list()
embedded_val_eval_emotion['label'] = val_eval_emotion['label'].to_list()

Starting to generate sentence embeddings


100%|█████████████████████████████████████████████████████████████████████████████| 3257/3257 [00:22<00:00, 142.50it/s]


Starting to generate sentence embeddings


100%|███████████████████████████████████████████████████████████████████████████████| 374/374 [00:01<00:00, 226.23it/s]


In [11]:
embedded_train_carer = fasttext.generate_sentence_embeddings(train_carer['text'])
embedded_val_carer = fasttext.generate_sentence_embeddings(val_carer['text'])
embedded_train_carer['label'] = train_carer['label'].to_list()
embedded_val_carer['label'] = val_carer['label'].to_list()

Starting to generate sentence embeddings


100%|███████████████████████████████████████████████████████████████████████████| 16000/16000 [00:45<00:00, 351.19it/s]


Starting to generate sentence embeddings


100%|█████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:06<00:00, 323.26it/s]


In [10]:
embedded_train_silicone = fasttext.generate_sentence_embeddings(train_silicone['text'])
embedded_val_silicone = fasttext.generate_sentence_embeddings(val_silicone['text'])
embedded_train_silicone['label'] = train_silicone['label'].to_list()
embedded_val_silicone['label'] = val_silicone['label'].to_list()

Starting to generate sentence embeddings


100%|███████████████████████████████████████████████████████████████████████████| 87170/87170 [01:51<00:00, 781.30it/s]


Starting to generate sentence embeddings


100%|█████████████████████████████████████████████████████████████████████████████| 8069/8069 [00:13<00:00, 600.92it/s]


## Hyperopt

In [6]:
import yaml
#Load the template yaml sweep config file for logistic regression
#If the value range for an hyperparameter needs to be changed, better to do it in the .yaml file than in a notebook
with open("config/svm_sweep.yaml", 'r') as stream:
    sweep_config = yaml.safe_load(stream)

In [9]:
#The config is displayed as a nested dictionary
sweep_config

{'method': 'random',
 'entity': 'benchmark-nlp',
 'project': 'hyperopt',
 'metric': {'name': 'loss', 'goal': 'minimize'},
 'parameters': {'C': {'min': 0, 'max': 10, 'distribution': 'uniform'},
  'kernel': {'values': ['linear', 'rbf']},
  'probability': {'value': True},
  'random_state': {'value': 42}}}

#### eval emotion

In [15]:
#Don't forget to name the sweep instance  
name = 'svm_tfidf_eval_emotion' #change here
sweep_config['name'] =  name

#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="svm")

def train_tfidf(config = None,
          train=train_eval_emotion, #Change here
          val=val_eval_emotion): #change here
    '''
    Generic WandB function to conduct hyperparameter optimization with tf-idf vectorizer
    '''
    # Initialize a new wandb run
    with wandb.init(config=config,group=name):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        vec = TfidfVectorizer()
        clf = SVC(C=config.C, 
                  kernel=config.kernel,
                  probability=config.probability,
                  random_state=config.random_state) #set the hyperparams here
        #Create the pipeline
        pipe = Pipeline([('vectorizer',vec),('clf',clf)])
        #Fit the pipeline
        pipe.fit(train['text'],train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val['text'])
        pred_prob_val = pipe.predict_proba(val['text'])[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
        else:
            aucpc = '-'
            auc = '-'
        #Log metrics on WandB
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })

#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_tfidf,count=OPT_ITER) #Count : number of iterations
tracker.stop()

Create sweep with ID: 3ea77bj6
Sweep URL: https://wandb.ai/benchmark-nlp/svm/sweeps/3ea77bj6


[codecarbon WARNING @ 17:48:03] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 17:48:06] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: 448e4xcl with config:
wandb: 	C: 9.423066515664598
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.66043
f1 macro,0.55692


wandb: Agent Starting Run: 6b9hffns with config:
wandb: 	C: 8.151009962757636
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.59358
f1 macro,0.50662


...

wandb: Agent Starting Run: zrhk3841 with config:
wandb: 	C: 8.345753913270567
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.65775
f1 macro,0.55519


wandb: Agent Starting Run: x6ajvmh4 with config:
wandb: 	C: 9.4060073807554
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.66043
f1 macro,0.55692


0.0015878593236364306

In [16]:
#Don't forget to name the sweep instance   
name = 'svm_ft_eval_emotion' #change here
sweep_config['name'] = name
#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="svm")

def train_fasttext(config = None,
          train=embedded_train_eval_emotion, #Change here
          val=embedded_val_eval_emotion): #change here
    # Initialize a new wandb run
    with wandb.init(config=config, group=name):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        clf = SVC(C=config.C, 
                  kernel=config.kernel,
                  probability=config.probability,
                  random_state=config.random_state) #set the hyperparams here
        pipe = Pipeline([('clf',clf)])
        pipe.fit(train.fillna(0).drop(['label'],axis=1),train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val.fillna(0).drop(['label'],axis=1))
        pred_prob_val = pipe.predict_proba(val.fillna(0).drop(['label'],axis=1))[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
            #Log predictions on WandB
        else:
            aucpc = '-'
            auc = '-'
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })


#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_fasttext,count=OPT_ITER)
tracker.stop()

Create sweep with ID: r0w06auo
Sweep URL: https://wandb.ai/benchmark-nlp/svm/sweeps/r0w06auo


[codecarbon WARNING @ 17:58:44] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 17:58:47] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: 808ttoyo with config:
wandb: 	C: 8.10364331881112
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.64973
f1 macro,0.57857


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 7rbmro3f with config:
wandb: 	C: 1.089636323090759
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.61765
f1 macro,0.47861


...

wandb: Agent Starting Run: kuhlys6r with config:
wandb: 	C: 2.911120222171768
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.6631
f1 macro,0.58445


wandb: Agent Starting Run: os701n1f with config:
wandb: 	C: 2.242588536733482
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.63102
f1 macro,0.53363


0.0020062178776431004

#### CARER

In [17]:
#Don't forget to name the sweep instance  
name = 'svm_tfidf_carer' #change here
sweep_config['name'] =  name

#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="svm")

def train_tfidf(config = None,
          train=train_carer, #Change here
          val=val_carer): #change here
    '''
    Generic WandB function to conduct hyperparameter optimization with tf-idf vectorizer
    '''
    # Initialize a new wandb run
    with wandb.init(config=config,group=name):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        vec = TfidfVectorizer()
        clf = SVC(C=config.C, 
                  kernel=config.kernel,
                  probability=config.probability,
                  random_state=config.random_state) #set the hyperparams here
        #Create the pipeline
        pipe = Pipeline([('vectorizer',vec),('clf',clf)])
        #Fit the pipeline
        pipe.fit(train['text'],train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val['text'])
        pred_prob_val = pipe.predict_proba(val['text'])[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
        else:
            aucpc = '-'
            auc = '-'
        #Log metrics on WandB
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })

#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_tfidf,count=OPT_ITER)
tracker.stop()

Create sweep with ID: lt2papke
Sweep URL: https://wandb.ai/benchmark-nlp/svm/sweeps/lt2papke


[codecarbon WARNING @ 18:12:15] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 18:12:18] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: zapnqcyf with config:
wandb: 	C: 7.796410356663496
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.8745
f1 macro,0.83782


wandb: Agent Starting Run: ryip2ka0 with config:
wandb: 	C: 1.8965406731910208
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.891
f1 macro,0.86241


...

wandb: Agent Starting Run: 4jxxzs3s with config:
wandb: 	C: 9.039720058600263
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.876
f1 macro,0.84721


wandb: Agent Starting Run: t2ywciwh with config:
wandb: 	C: 7.1701053138700575
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.8795
f1 macro,0.84981


0.008614541476856117

In [18]:
#Don't forget to name the sweep instance   
name = 'svm_ft_carer' #CHANGE HERE
sweep_config['name'] = name
#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="svm")

def train_fasttext(config = None,
          train=embedded_train_carer, #CHANGE HERE
          val=embedded_val_carer): #CHANGE HERE
    # Initialize a new wandb run
    with wandb.init(config=config, group=name):
        config = wandb.config
        clf = SVC(C=config.C, 
                  kernel=config.kernel,
                  probability=config.probability,
                  random_state=config.random_state) #set the hyperparams here
        pipe = Pipeline([('clf',clf)])
        pipe.fit(train.fillna(0).drop(['label'],axis=1),train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val.fillna(0).drop(['label'],axis=1))
        pred_prob_val = pipe.predict_proba(val.fillna(0).drop(['label'],axis=1))[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
            #Log predictions on WandB
        else:
            aucpc = '-'
            auc = '-'
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })


#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_fasttext,count=OPT_ITER)
tracker.stop()

Create sweep with ID: uguo86hy
Sweep URL: https://wandb.ai/benchmark-nlp/svm/sweeps/uguo86hy


[codecarbon WARNING @ 19:09:42] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 19:09:48] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: nys6p7au with config:
wandb: 	C: 6.831610531306077
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.671
f1 macro,0.55779


wandb: Agent Starting Run: hgdz65gc with config:
wandb: 	C: 1.8923282168886535
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.6775
f1 macro,0.56607


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: gg3ip5il with config:
wandb: 	C: 7.871551473979233
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.6715
f1 macro,0.56288


wandb: Agent Starting Run: o5fcliw2 with config:
wandb: 	C: 9.533113109707688
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.673
f1 macro,0.56968


0.015461782987135905

#### silicone

In [7]:
#Don't forget to name the sweep instance  
name = 'svm_tfidf_silicone' #change here
sweep_config['name'] =  name

#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="svm")

def train_tfidf(config = None,
          train=train_silicone, #Change here
          val=val_silicone): #change here
    '''
    Generic WandB function to conduct hyperparameter optimization with tf-idf vectorizer
    '''
    # Initialize a new wandb run
    with wandb.init(config=config,group=name):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        vec = TfidfVectorizer()
        clf = LinearSVC(C=config.C, 
                        loss='hinge',
#                   kernel=config.kernel,
#                   probability=config.probability,
                  random_state=config.random_state) #set the hyperparams here
        #Create the pipeline
        pipe = Pipeline([('vectorizer',vec),('clf',clf)])
        #Fit the pipeline
        pipe.fit(train['text'],train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val['text'])
#         pred_prob_val = pipe.predict_proba(val['text'])[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
#         if train['label'].nunique() <=2:
#             aucpc =  average_precision_score(val['label'],pred_prob_val)
#             auc = roc_auc_score(val['label'],pred_prob_val)
#         else:
        aucpc = '-'
        auc = '-'
        #Log metrics on WandB
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })

#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_tfidf, count=OPT_ITER)
tracker.stop()

Create sweep with ID: 35hdqhr1
Sweep URL: https://wandb.ai/benchmark-nlp/svm/sweeps/35hdqhr1


[codecarbon WARNING @ 12:14:57] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 12:15:00] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: unrw1945 with config:
wandb: 	C: 2.145705009196047
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.89193
f1 macro,0.26193


wandb: Agent Starting Run: ypwjssoe with config:
wandb: 	C: 1.4196474004701476
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.89354
f1 macro,0.25569


...

wandb: Agent Starting Run: 7xjyez2h with config:
wandb: 	C: 7.305191251744239
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.88797
f1 macro,0.27584


wandb: Agent Starting Run: ovv40dhv with config:
wandb: 	C: 6.779042487218171
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.88846
f1 macro,0.27648


0.001157240238882834

In [11]:
#Don't forget to name the sweep instance   
name = 'svm_ft_silicone' #CHANGE HERE
sweep_config['name'] = name
#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="svm")

def train_fasttext(config = None,
          train=embedded_train_silicone, #CHANGE HERE
          val=embedded_val_silicone): #CHANGE HERE
    # Initialize a new wandb run
    with wandb.init(config=config, group=name):
        config = wandb.config
        clf = LinearSVC(C=config.C, 
                  loss='hinge',
#                   kernel=config.kernel,
#                   probability=config.probability,
                  random_state=config.random_state) #set the hyperparams here
        pipe = Pipeline([('clf',clf)])
        pipe.fit(train.fillna(0).drop(['label'],axis=1),train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val.fillna(0).drop(['label'],axis=1))
#         pred_prob_val = pipe.predict_proba(val.fillna(0).drop(['label'],axis=1))[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
#         if train['label'].nunique() <=2:
#             aucpc =  average_precision_score(val['label'],pred_prob_val)
#             auc = roc_auc_score(val['label'],pred_prob_val)
#             #Log predictions on WandB
#         else:
        aucpc = '-'
        auc = '-'
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })


#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_fasttext, count=OPT_ITER)
tracker.stop()

Create sweep with ID: jz1vlwnl
Sweep URL: https://wandb.ai/benchmark-nlp/svm/sweeps/jz1vlwnl


[codecarbon WARNING @ 12:47:22] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 12:47:25] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: ymeo00tf with config:
wandb: 	C: 6.0466256908660725
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.88251
f1 macro,0.15524


wandb: Agent Starting Run: yvvmo4mf with config:
wandb: 	C: 1.7868754545190857
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.88264
f1 macro,0.15649


...

wandb: Agent Starting Run: 1t7qiuo3 with config:
wandb: 	C: 7.247940091193216
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.88251
f1 macro,0.15615


wandb: Agent Starting Run: pbzpa5da with config:
wandb: 	C: 2.1106469446999787
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.88239
f1 macro,0.1552


0.0024874125108176118